# A111 RRAM区块操作

## crossbar(XB) 基本信息

每个TILE含有8个xb，每两个xb可以独立工作，也可以组合进行计算；

XB阵列模块为320行128列的2T2R阵列：

<div><a href="pic/xb1.png"><img style="float: left;" src="pic/xb1.png" width=300 title="点击放大看原图"></a></div>

<div><a href="pic/xb2.png"><img style="float: left;" src="pic/xb2.png" width=600 title="点击放大看原图"></a></div>

In [1]:
import numpy as np
import pandas as pd
import a111sdk

In [2]:
# 给定csv路径，读取数据
def pd_read_data(data_path):
    data = pd.read_csv(data_path, header=None, index_col=None).values
    return data

# 给定pt形式的数组，制作rram格式的数组，即pt顺序的格式转为rram4i+j的格式
def pt_sequence_2_rram_discretization(pt_sequence):
    pt_sequence_row, pt_sequence_colum = pt_sequence.shape
    rram_discretization = np.zeros([pt_sequence_row, 128])
    pt_sequence_128colum = np.zeros([pt_sequence_row, 128])
    pt_sequence_128colum[:, :pt_sequence_colum] = pt_sequence
    # 遍历127次，对应索引为：pt0-rram0,pt1-rram4,pt31-rram124,pt32-rram1,pt126-rram123,pt127-rram127
    for rram_colum in range(127):
        mapping_index = (4 * rram_colum) % 127
        rram_discretization[:, mapping_index] = pt_sequence_128colum[:, rram_colum]
    # 最后一次需要单独赋值，pt127-rram127
    rram_discretization[:, 127] = pt_sequence_128colum[:, 127]
    return rram_discretization


# 给定pt形式的权重，转为rram需要的权重，包括具体值大小和格式
def trans_pt_weight_2_rram(pt_weight):
    # 对于pt的3值权重，映射到rram上需要改变具体值，也就是rram = pt x pos_sa 或者rram = pt x neg_sa
    pos_sa = 5
    neg_sa = 5

    row, colum = pt_weight.shape
    # 转换原始pt权重为2T2R权重
    rram_weight = np.zeros([row * 2, colum])
    pos_weight = np.zeros_like(pt_weight)
    neg_weight = np.zeros_like(pt_weight)
    flag = pt_weight > 0
    pos_weight[flag] = pos_sa
    flag = pt_weight < 0
    neg_weight[flag] = neg_sa
    rram_weight[::2, :] = pos_weight
    rram_weight[1::2, :] = neg_weight
    # 根据芯片mapping策略，重构rram权重（每隔4列存一个数据，满列操作，即128列都用）
    sub_mapping_weight = pt_sequence_2_rram_discretization(rram_weight)
    # 补全其余行的数据，最终芯片mapping的权重需求为640x128的矩阵
    mapping_weight = np.zeros([640, 128])
    mapping_weight[:rram_weight.shape[0]] = sub_mapping_weight
    mapping_weight = mapping_weight.astype(np.uint8)
    return mapping_weight

# 给定rram格式的权重、tile、xb，自动mapping权重
def auto_mapping_weight(pt_weight_2_rram, tile=0, xb=0):
    rram_weight_demo = pt_weight_2_rram
    row_begin, row_length, colum_begin, colum_length = [0, rram_weight_demo.shape[0], 0, rram_weight_demo.shape[1]]
    index = [row_begin, colum_begin, row_length, colum_length]
    # rram_weight_demo 大小为32x32，index为 [0, 32, 0, 32]
    a111sdk.a111_mapping_weight(rram_weight_demo, tile_id=tile, xb_id=xb, addr=index)

## 写入RRAM值

In [4]:
a111sdk.open_a111()

pt_weight = pd_read_data("data/fc1_weight.csv")
pt_weight_2_rram = trans_pt_weight_2_rram(pt_weight)
auto_mapping_weight(pt_weight_2_rram, tile=a111sdk.TILE0, xb=a111sdk.XB0)

a111sdk.close_a111()

设备已打开！
weight max :5
weight min :0
weight shape:(640, 128)
硬件初始化成功！！！
时钟初始化成功！！！
开始 program 第 0 次======>
map cnt:[000],pass:77366,check:00000,set:04544,reset:00010,form:00000,final:00000,Total:81920
map cnt:[001],pass:78427,check:00005,set:03473,reset:00015,form:00000,final:00000,Total:81920
map cnt:[002],pass:78211,check:00391,set:03317,reset:00001,form:00000,final:00000,Total:81920
map cnt:[003],pass:01019,check:00395,set:03144,reset:00001,form:00000,final:77361,Total:81920
map cnt:[004],pass:00522,check:00148,set:03293,reset:00000,form:00000,final:77957,Total:81920
map cnt:[005],pass:00658,check:00156,set:03013,reset:00000,form:00000,final:78093,Total:81920
map cnt:[006],pass:00699,check:00135,set:02863,reset:00000,form:00000,final:78223,Total:81920
map cnt:[007],pass:00670,check:00133,set:02779,reset:00000,form:00000,final:78338,Total:81920
map cnt:[008],pass:00493,check:00107,set:02708,reset:00000,form:00000,final:78612,Total:81920
map cnt:[009],pass:00392,check:00098,set:02645,re

map cnt:[087],Program times : 1, Program cost time : 184.28397512435913s
设备已关闭！


## 读取RRAM值

In [5]:
a111sdk.open_a111()
rram_read_weight = a111sdk.a111_read_weight(a111sdk.TILE0, a111sdk.XB0)
print(rram_read_weight)
a111sdk.close_a111()

设备已打开！
硬件初始化成功！！！
时钟初始化成功！！！
开始 read Tile[0].XB[0].[0,640,0,128] ======>
tile[0].xb[0]: 
    weight max: 7
    weight min: 0
    weight mean: 0.22076416015625
[[1 2 1 ... 0 1 0]
 [0 0 1 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
设备已关闭！
